In [4]:
!pip install pymorphy2
!pip install razdel
!pip install rusenttokenize

In [5]:
!pip install pymystem3

# Домашнее задание № 1

## 1. Задание на удаление тегов (2 балла)

Часто в данных, с которыми нам нужно работать помимо текста присутствует ещё какая-то лишняя информация: тэги, ссылки, код, разметка. Она, конечно, не всегда лишняя, но обычно от неё лучше избавиться.

Возьмем в качестве примера статью с Хабрахабра. Она скачана автоматически и в там остались некоторые тэги.

In [6]:
import string
from rusenttokenize import ru_sent_tokenize
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from nltk.corpus import stopwords
from string import punctuation
import re
import os
import json

In [7]:
text = '''
Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!<br><br><img src="http://img172.imageshack.us/img172/8434/18274658kc4.png" alt="Сумасшедшее яйцо"><br><br><blockquote><h3>Что это такое?</h3><br>
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.<br>
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).<br><br><img src="https://habrastorage.org/getpro/habr/post_images/7c7/298/33c/7c729833cd942cc493e68833e3e0f12d.jpg" alt="Тепловое отслеживание популярности"><br></blockquote><br><br><a name="habracut"></a><br><br><blockquote><h3>Для кого это?</h3><br>
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн дает о себе знать — сервис массово популяризировался пол года-год назад. Относительно недорогой, но уж точно не из дешевых — сервис предполагает собой 4 платных линии и одну бесплатную. Дабы написать этот обзор я не поленился заплатить 19 долларов (в месяц) выбрав средний вариант — для нескольких проектов с включенными дополнительными функциями.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0b0/0a0/b16/0b00a0b16b1eda28e35f39487dcd2545.jpg" alt="Отслеживание ссылок"></blockquote><br><blockquote><img src="https://habrastorage.org/getpro/habr/post_images/0b5/433/892/0b54338921665ffb5a90930147296f5b.jpg" alt="Список по популярности"><br><br>
Да-да, сервис не бесплатен. Точнее бесплатная возможность потестировать есть, но она немного обрезана (можно отслеживать только 5000 посетителей и всего 4 страницы на сайте (внимание — 4 страницы, а не сайтов), т.е. вполне хватает для того, чтобы понять полезность сервиса). Зачем это?</h3><br>
С помощью этого сервиса можно тасовать блоки на сайте, которые полезны пользователям больше всего. Больше не нужно спорить создателям — какой блок где расположить. В этом им поможет <a href="http://ttp://www.crazyegg.com" title="Сумасшедшие яйца">Сумасшедшее яйцо</a>.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/38c/af8/e75/38caf8e753782a01dc6419d3902edd57.jpg" alt="Добавление проекта"></blockquote><br><blockquote>Также этот сервис поможет вам понять — в какой зоне сайта лучше всего располагать рекламу, когда в вашем сервисе речь зайдет о монетизации. Ведь альтруизм это хорошо, а деньги на содержание сервиса нужны, и не лишним будет вычислить зоны где реклама будет приносить наибольшую отдачу, и наименьшее раздражение у пользователей.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0dd/3fb/b34/0dd3fbb34709e77f3fbcd6523c7eac77.jpg" alt="еще полезности"></blockquote><br><blockquote><h3>Как это работает?</h3><br>
Никаких километровых скриптов вставлять не нужно, достаточно вставиь 2 строчки яваскрипта, и сервис начнет отслеживание. Насколько я понял — исполнительный скрипт работает на сервере <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>, поэтому ваш сайт от этого в производительности не потеряет ни секунды, а полезность довольно таки большая.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/a90/e4e/e67/a90e4ee6760978463e6306a2f5982e24.jpg" alt="управление популярностью"></blockquote><br><blockquote><h3>Сколько это стоит?</h3><br>
Как я уже упоминал — сервис далеко не бесплатен, хоть и имеет тестовую-бесплатную версию. Расскажу подробнее о тарификации.</blockquote><br><blockquote>1. Бесплатная тестовая версия. Включает в себя возможность отслеживания 5 000 посещений, 4 страницы на сайте.<br><br>
2. Базовая версия. В этой версии можно отследить 10 000 посещений, и 10 страниц, что вполне достаточно для среднего корпоративного сайта. Стоит базовый комплект — 9 долларов в месяц. В этот комплект включены все дополнительные функции.<br><br>
3. Версия «Стандарт». В неё входит возможность отслеживания 25 000 посещений на 20 страницах. Вполне подходит для тестирования нового стартапа. Стоит она 19 долларов в месяц, именно её я купил для тестирования сервиса, и написания этого обзора.<br><br>
4. Версия «Плюс». Отличается от предыдущей возможностью отслеживания 100 000 посещений, 50 страниц. Очень хороший тариф для крупных сервисов. Стоит 49 долларов в месяц. Довольно большие деньги за сервис, но они обычно с лихвой окупаются.<br><br>
5. Версия «Про». Стоит почти 100 долларов, имеет возможность отследить 250 000 посещений на ста страницах. Тариф подходит для монстров с большой посещаемостью и большим количеством страниц.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/f73/fb2/62d/f73fb262da4618a8dde67690cfd191ea.jpg" alt="Отслеживание статистики"></blockquote><br><blockquote><h3>Есть и аналоги</h3><br>
Я не поленился, и собрал еще пару ссылок с аналогами, которые предлагают такие же услуги, но немного дешевле.</blockquote><br><blockquote> 1. <a href="https://www.google.com/analytics/home/?hl=en" title="Шикарный сбор и анализ статистики">Google Analytics</a> — бесплатный сервис для сбора и анализа статистики, вывод статистики в наиболее наглядной форме, и без разнообразных рейтингов, счетчиков. Очень подробная и полезная вещь. Рекомендую, т.к. сам пользуюсь им для этого блога.<br><br>
2. <a href="http://www.mapsurface.com/" title="Сервис для отслеживания популярности блоков на сайте">MapSurface</a> — сам еще не использовал (т.к. предпочел CrazyEgg), но врядли будучи плохим сервис собрал бы множество положительных отзывов. К сожалению он сейчас находится в статусе закрытой беты.</blockquote><br><blockquote><h3>Вывод</h3><br>
Использовать можно, и нужно. Вот только тарифы довольно больно кусаются, но обычно эти деньги потом с лихвой отбиваются на повышении конвертации посетителей в деньги. Использовать сервис нужно для тестирования рекламных мест и удобства отдельных страниц, что помогает опять же повысить конвертацию. В общем и целом — <a href="http://ttp://www.crazyegg.com">полезная вещь</a> для каждого владельца сайтов, а для юзабилиста вообще практически обязательна. К счастью для людей, которые поиздержались деньгами в этом месяце — есть полезные аналоги.<br><br><img src="http://img241.imageshack.us/img241/2346/confettitnml5.jpg" alt="Интересная идея визуализации популярности - конфети"></blockquote><br><br>
Автор: <a href="http://www.birzool.com/" title="Я пишу о юзабилити веб интерфейсов">Ярослав Бирзул</a> (DezmASter).<br>
Источник: <a href="http://www.birzool.com/crazyegg/" title="Сумасшедшие яйца, первоисточник">Блог о юзабилити веб интерфейсов</a>.<br><br>
PS: Всех с прошедшим Новым годом! От всей души желаю вам всего самого-самого лучшего, чего вы желаете только в самых сокровенных мечтах. Удачно вам провести время.
'''

In [8]:
def remove_tags(text):
    return re.sub(r'<[^>]+>', '', text)

In [9]:
cleaned_text = remove_tags(text)

In [10]:
print(cleaned_text)


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!Что это такое?
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).Для кого это?
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн да

С помощью регулярных выражение очистите текст от тэгов (мета информацию внутри тэгов тоже удалите). В получившемся тексте точки не должны слипаться с началом следующего предложений, также не должно быть лишних пробелов.

## 2. Задание на выбор слов после "не " (1 балл)

Вытащите из очищенного текста все слова, которые идут после частицы "не" (случаи, когда "не" - часть предыдущего слова, не должны извлекаться), **кроме слов, начинающихся на гласную букву**. Сама часть "не " не должна извлекаться.

In [11]:
re.findall(r'\s[Нн]е\s([БбВвГгДдЖжЗзЙйКкЛлМмНнПпРрСсТтФфХхЦцЧчШшЩщ]\w+)', cleaned_text)

['знаю',
 'поленился',
 'бесплатен',
 'сайтов',
 'нужно',
 'лишним',
 'нужно',
 'потеряет',
 'бесплатен',
 'поленился']

## 3. Задание на выбор глаголов (1 балл)

Попробуйте вытащить из очищенного текста как можно больше глаголов (минимум 40 с повоторениями), не перечисляя их в паттерне полностью. Можете токенизировать текст и использовать re.match, а можете использовать re.findall с \\b в начале и конце паттерна, чтобы не матчить части слов.

В результате не должно быть не глаголов. Также постарайтесь сделать паттерн максимально коротким.

In [12]:
v = re.findall(r'\b\w+[ауюеи]ть?с?я?\b', cleaned_text)
print(v)

['будет', 'работает', 'Запоминается', 'позволяет', 'отслеживать', 'выводить', 'совмещает', 'Разумеется', 'пользоваться', 'может', 'дает', 'знать', 'предполагает', 'написать', 'заплатить', 'потестировать', 'отслеживать', 'хватает', 'тасовать', 'спорить', 'расположить', 'поможет', 'поможет', 'располагать', 'зайдет', 'будет', 'вычислить', 'будет', 'приносить', 'работает', 'начнет', 'работает', 'потеряет', 'стоит', 'имеет', 'Включает', 'отследить', 'Стоит', 'входит', 'подходит', 'Стоит', 'Отличается', 'Стоит', 'окупаются', 'Стоит', 'имеет', 'отследить', 'подходит', 'предлагают', 'находится', 'Использовать', 'кусаются', 'отбиваются', 'Использовать', 'помогает', 'повысить']


In [13]:
print(len(v))

56


## 4. Pymorphy vs Mystem (2 балла)

Лемматизируйте очищенный текст с помощью mystem и pymorphy. Сравните результаты: есть ли разница, что лучше. 

Для пайморфи используйте токенизацию из mystem, чтобы исключить влияние токенизации на результат.

In [18]:
mystem = Mystem()
morph = MorphAnalyzer()

In [28]:
lemmas_mystem = mystem.lemmatize(cleaned_text)
print(''.join(lemmas_mystem))


сегодняшний заметка быть о сервис отслеживание активность пользователь — CrazyEgg. я не знать кто обязанный сервис такой говорящий имя, но оно работать, и хорошо. запоминаться? отлично!что это такой?
как уже сказать выше это сервис для отслеживание перемещение пользователь по сайт — кто куда кликнуть, какой ссылка наиболее популярный и тот подобный, разнородный информация для юзабилист.
сервис позволять отслеживать активность определенный пользователь, и выводить этот данные в различный форма: «инфракрасный» — где чем активно область, тем она «тепло», салют (чем активно область, тем много конфеть), простой список с сортировка по активность, колба (чем заполненный колба, тем активно область), облако (на мой взгляд наиболее удобный вариант — совмещать в себя весь остальной вместе взять).для кто это?
разумеется сервис изначально планироваться как первый помощник для юзабилист, но пользоваться они может каждый, у кто быть деньги — интуитивный интерфейс, и хороший дизайн давать о себя знат

В целом Mystem работает нормально, но есть небольшие проблемы:
"Конфети" лемматизировалось как "конфеть"
"Стоять" как "стоить"
"Есть" как "быть"

Также есть небольшие недостатки в принципах, по которым Mystem лемматизирует слова:
Глаголы совершенного вида преобразуются в инфинитивы несовершенного ("вычислит" - "вычислять", "начнет" - "начинать")
Прилагательные в сравнительной степени Mystem лемматизирует в наречия ("активнее" - "активный", "больше" - "большой")

In [33]:
lemmatized_pymorphy = []
analyzed = mystem.analyze(cleaned_text)

for token in analyzed:
    parsed = morph.parse(token['text'])[0]
    lemmatized_pymorphy.append(parsed.normal_form)
lemmas_pymorphy = "".join(lemmatized_pymorphy)
print(lemmas_pymorphy)


сегодняшний заметка быть о сервис отслеживание активность пользователь — crazyegg. я не знать кто обязать сервис такой говорящий имя, но оно работать, и хорошо. запоминаться? отлично!что это такой?
как уже сказать выше это сервис для отслеживание перемещение пользователь по сайт — кто куда кликнуть, какой ссылка наиболее популярный и тот подобный, разнородный информация для юзабилист.
сервис позволять отслеживать активность определённый пользователь, и выводить этот дать в различный форма: «инфракрасный» — где чем активный область, тем она «тёплый», салют (чем активный область, тем большой конфести), простой список с сортировка по активность, колба (чем залненный колба, тем активный область), облако (на мой взгляд наиболее удобный вариант — совмещать в себя всё остальной вместе взять).для кто это?
разумеется сервис изначально планироваться как первый помощник для юзабилист, но пользоваться они мочь каждый, у кто есть деньга — интуитивный интерфейс, и хороший дизайн давать о себя знать

PyMorphy, кажется, работает похуже. К проблемам с некорректной лемматизацией некоторых слов, добавились также некоторые другие:
"дабы" - "даба"
"врядлить" - "врядли"
Лемматизация глаголов и сравнительных степеней прилагательных в PyMorphy реализована корректно. Хуже реализована лемматизация заимствований: "юзабилиста" – "юзабилистый", "юзабилити" – юзабилить"

## 5. Разбиение на предложения в Mystem (2 балла)

С помощью Mystem разделите текст из первого задания (очищенный) на предложения. Сравните разбиение с razdel.sentenize

In [40]:
with open('cleaned_text.txt', 'w', encoding = 'UTF-8') as f:
    f.write(cleaned_text)

/bin/bash: mystem: command not found


In [48]:
import json
sents = [json.loads(line) for line in open('../input/homework2/text_parsed.txt', encoding = 'UTF-8')]
sent = ''
for par in sents:
    for n in par:
        sent+=n['text']
mystem_sents = sent.split('\\s')
print(mystem_sents)

['\nСегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. ', 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. ', 'Запоминается? ', 'Отлично!Что это такое?\nКак уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.\nСервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).Для кого это?\nРазумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и

In [50]:
razdel_sents = [parse.text for parse in list(sentenize(cleaned_text))]
print(razdel_sents)

['Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg.', 'Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.', 'Запоминается?', 'Отлично!Что это такое?', 'Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.', 'Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).Для кого это?', 'Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, 

Razdel работает получше, в MyStem я вижу две проблемы: программа не различает символы переноса и оставляет пробелы в конце предложений